# Use trained embeddings in downstream tasks
You can use the trained Word2Vec/ Fasttext/ Glove embeddings as input features for various downstream NLP tasks such as text classification, named entity recognition, or machine translation. Here's a simple example of how to use the embeddings for text classification using a logistic regression model.

In [3]:
from nltk.tokenize import sent_tokenize, word_tokenize


Go to Kaggle website and download the archive "Reviews.csv" from the dataset "Amazon Fine Food Reviews": https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews

In [4]:
import pandas as pd
rev = pd.read_csv("Reviews.csv")
print(rev.head())

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

# We create the list of the words that our corpus has

In [5]:
corpus_text = 'n'.join(rev[:1000]['Text'])
data = []
# iterate through each sentence in the file
for i in sent_tokenize(corpus_text):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [6]:
import re
from tqdm import tqdm
import numpy as np
from pathlib import Path


In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("leadbest/googlenewsvectorsnegative300")

print("Path to dataset files:", path)

/Users/bilgesipal/PycharmProjects/boun_MIS48D/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/bilgesipal/.cache/kagglehub/datasets/leadbest/googlenewsvectorsnegative300/versions/2


In [8]:
path

'/Users/bilgesipal/.cache/kagglehub/datasets/leadbest/googlenewsvectorsnegative300/versions/2'

In [9]:
emb_path = list(Path(path).rglob("GoogleNews-vectors-negative300.bin"))[0]
print("Embedding file:", emb_path)

Embedding file: /Users/bilgesipal/.cache/kagglehub/datasets/leadbest/googlenewsvectorsnegative300/versions/2/GoogleNews-vectors-negative300.bin


In [12]:
def load_word2vec_binary(path, limit=None):
    """Load GoogleNews .bin Word2Vec file (no gensim)."""
    with open(path, "rb") as f:
        header = f.readline()
        vocab_size, dim = map(int, header.split())
        print(f"Header: {vocab_size} words, dim={dim}")
        word_vectors = {}
        for i in tqdm(range(vocab_size if not limit else min(limit, vocab_size))):
            # Read word
            word = []
            while True:
                ch = f.read(1)
                if ch == b' ':
                    break
                if ch == b'':
                    break
                word.append(ch)
            word = b''.join(word).decode("utf-8", errors="ignore")
            # Read vector
            vec = np.frombuffer(f.read(dim * 4), dtype=np.float32)
            f.read(1)  # newline
            word_vectors[word] = vec
        return word_vectors, dim

In [13]:
#  Load limited subset for demo
w2v, dim = load_word2vec_binary(emb_path, limit=200000)
print("Loaded words:", len(w2v))

Header: 3000000 words, dim=300


100%|██████████| 200000/200000 [00:01<00:00, 181214.23it/s]

Loaded words: 152943


# The simplest representation of documents is avarage of word vectors

In [14]:
def tokenize(text):
    return re.findall(r"[A-Za-z']+", text.lower())


In [15]:
corpus_text = '\n'.join(rev.loc[:999, 'Text'].astype(str))
data = []

for i in sent_tokenize(corpus_text):
    temp = [j.lower() for j in word_tokenize(i)]
    data.append(temp)

# --- 4. Labels ---
labels = rev.loc[:999, 'Score'].astype(str).tolist()
print("Number of samples:", len(data))
print("Number of labels:", len(labels))

Number of samples: 4511
Number of labels: 1000


In [16]:
def doc_vector(tokens, w2v, dim):
    vecs = [w2v[t] for t in tokens if t in w2v]
    if not vecs:
        return np.zeros(dim)
    return np.mean(vecs, axis=0)

In [17]:
X = np.vstack([doc_vector(tokens, w2v, dim) for tokens in data[:1000]])
y = np.array(labels)

print("Feature matrix shape:", X.shape)

Feature matrix shape: (1000, 300)


In [18]:
len(y)

1000

# Example classification task

In [19]:
from sklearn.model_selection import train_test_split
#  Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y if len(set(y)) > 1 else None
)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report


#Train logistic regression
clf = LogisticRegression(max_iter=200, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

#Evaluate
labels_for_cm = sorted(set(y_test) | set(y_pred))  # safe union of actual and predicted

print(classification_report(y_test, y_pred,
                            labels=labels_for_cm,
                            zero_division=0))


              precision    recall  f1-score   support

           1       0.00      0.00      0.00        29
           2       0.00      0.00      0.00        14
           3       0.00      0.00      0.00        23
           4       0.00      0.00      0.00        41
           5       0.64      0.99      0.78       193

    accuracy                           0.64       300
   macro avg       0.13      0.20      0.16       300
weighted avg       0.41      0.64      0.50       300

